Imports

In [ ]:
import pandas as pd
from datetime import datetime
from CloudBucket import CloudBucket
import httpx
import io
from decouple import config
import pandas as pd
import io
import json


Loading the Dataframe

In [ ]:
server_location = 'obs.sa-brazil-1.myhuaweicloud.com'
access_key_id = 'UUPFWV82Q3J8VYVHNZBQ'
secret_access_key = 'Ib4bDYbx5jdLrjDQulpiQRSKGPer5ffkx5ppi9ug'
with CloudBucket(access_key_id=access_key_id, secret_access_key=secret_access_key, server_location=server_location) as client:
    bytes_file_response = client.get_binary_file(bucketName='test-datalake', file_name='uber_data.csv')
    df = pd.read_csv(io.BytesIO(bytes_file_response.body.buffer), sep=',')
df.head()

In [5]:
df = pd.read_csv(r'C:\Users\g50034179\Documents\studies\uber_analytics\assets\uber_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   VendorID               100000 non-null  int64  
 1   tpep_pickup_datetime   100000 non-null  object 
 2   tpep_dropoff_datetime  100000 non-null  object 
 3   passenger_count        100000 non-null  int64  
 4   trip_distance          100000 non-null  float64
 5   pickup_longitude       100000 non-null  float64
 6   pickup_latitude        100000 non-null  float64
 7   RatecodeID             100000 non-null  int64  
 8   store_and_fwd_flag     100000 non-null  object 
 9   dropoff_longitude      100000 non-null  float64
 10  dropoff_latitude       100000 non-null  float64
 11  payment_type           100000 non-null  int64  
 12  fare_amount            100000 non-null  float64
 13  extra                  100000 non-null  float64
 14  mta_tax                100000 non-nul

In [ ]:
df.dtypes.tolist()

In [ ]:
df.index.name = "Id"
df

dataFrame type -> transform to datatime

In [ ]:
df.info()

In [7]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

def test_df_datetype(df: pd.DataFrame) -> None:
    assert all([data_type == 'datetime64[ns]' for data_type in df[['tpep_pickup_datetime', 'tpep_dropoff_datetime']].dtypes])

test_df_datetype(df)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   VendorID               100000 non-null  int64         
 1   tpep_pickup_datetime   100000 non-null  datetime64[ns]
 2   tpep_dropoff_datetime  100000 non-null  datetime64[ns]
 3   passenger_count        100000 non-null  int64         
 4   trip_distance          100000 non-null  float64       
 5   pickup_longitude       100000 non-null  float64       
 6   pickup_latitude        100000 non-null  float64       
 7   RatecodeID             100000 non-null  int64         
 8   store_and_fwd_flag     100000 non-null  object        
 9   dropoff_longitude      100000 non-null  float64       
 10  dropoff_latitude       100000 non-null  float64       
 11  payment_type           100000 non-null  int64         
 12  fare_amount            100000 non-null  float

In [ ]:
df.dtypes

Dropping the duplicates and assing a PK to 'trip_id'

In [ ]:
df = df.drop_duplicates().reset_index(drop=True)
df['trip_id'] = df.index
df.head()

def test_trip_id(df: pd.DataFrame) -> None:
    def check_numeric_progression(id_list):
        initial_value = id_list[0]
        for idx, value in enumerate(id_list):
            if value != initial_value + idx:
                return False
        return True

    assert all(isinstance(id, int) for id in df['trip_id'])
    assert check_numeric_progression(df['trip_id'])

test_trip_id(df)
df.columns

Modeling the datetime_dim schema with pandas & testing the operation

In [ ]:
datetime_dim = df[['tpep_pickup_datetime', 'tpep_dropoff_datetime']].reset_index(drop=True)
datetime_dim['tpep_pickup_datetime'] = datetime_dim['tpep_pickup_datetime']
datetime_dim['pick_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['pick_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['pick_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['pick_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['pick_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday

datetime_dim['tpep_dropoff_datetime'] = datetime_dim['tpep_dropoff_datetime']
datetime_dim['drop_hour'] = datetime_dim['tpep_dropoff_datetime'].dt.hour
datetime_dim['drop_day'] = datetime_dim['tpep_dropoff_datetime'].dt.day
datetime_dim['drop_month'] = datetime_dim['tpep_dropoff_datetime'].dt.month
datetime_dim['drop_year'] = datetime_dim['tpep_dropoff_datetime'].dt.year
datetime_dim['drop_weekday'] = datetime_dim['tpep_dropoff_datetime'].dt.weekday

datetime_dim['datetime_id'] = datetime_dim.index
datetime_dim.head(20)

def test_datetime_dim(datatime_dim: pd.DataFrame) -> None:
    assert not datatime_dim.isnull().values.any()
    assert all([len(str(item)) == 4 and isinstance(item, int) for item in datetime_dim['pick_year']])
    assert all([isinstance(date, datetime) for date in datetime_dim['tpep_pickup_datetime']])
    assert all([isinstance(item, int) for item in datetime_dim['pick_hour']])
    assert all([isinstance(item, int) for item in datetime_dim['pick_day']])
    assert all([isinstance(item, int) for item in datetime_dim['pick_month']])
    assert all([isinstance(item, int) for item in datetime_dim['pick_year']])
    assert all([isinstance(item, int) for item in datetime_dim['pick_weekday']])

    assert all([len(str(item)) == 4 and isinstance(item, int) for item in datetime_dim['drop_year']])
    assert all([isinstance(date, datetime) for date in datetime_dim['tpep_dropoff_datetime']])
    assert all([isinstance(item, int) for item in datetime_dim['drop_hour']])
    assert all([isinstance(item, int) for item in datetime_dim['drop_day']])
    assert all([isinstance(item, int) for item in datetime_dim['drop_month']])
    assert all([isinstance(item, int) for item in datetime_dim['drop_year']])
    assert all([isinstance(item, int) for item in datetime_dim['drop_weekday']])

datetime_dim = datetime_dim[['datetime_id', 'tpep_pickup_datetime', 'pick_hour', 'pick_day', 'pick_month', 'pick_year', 'pick_weekday',
                             'tpep_dropoff_datetime', 'drop_hour', 'drop_day', 'drop_month', 'drop_year', 'drop_weekday']]

test_datetime_dim(datetime_dim)
datetime_dim['tpep_pickup_datetime'] = datetime_dim['tpep_pickup_datetime'].dt.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
datetime_dim['tpep_dropoff_datetime'] = datetime_dim['tpep_dropoff_datetime'].dt.strftime('%Y-%m-%dT%H:%M:%S.%fZ')




datetime_dim.head()
# datetime_dim['']



Modeling the passenger_count_dim schema with pandas & testing it

In [ ]:
passenger_count_dim = df[['passenger_count']].reset_index(drop=True)
passenger_count_dim['passenger_count_id'] = passenger_count_dim.index
passenger_count_dim = passenger_count_dim[['passenger_count_id', 'passenger_count']]

def test_passenger_count_dim(passenger_count_dim: pd.DataFrame) -> None:
    assert not passenger_count_dim.isnull().values.any()
    assert all([isinstance(item, int) for item in passenger_count_dim['passenger_count']])
    assert all([isinstance(item, int) for item in passenger_count_dim['passenger_count_id']])    

test_passenger_count_dim(passenger_count_dim)
passenger_count_dim.head()

In [ ]:
trip_distance_dim = df[['trip_distance']].reset_index(drop=True)
trip_distance_dim['trip_distance_id'] = trip_distance_dim.index
trip_distance_dim = trip_distance_dim[['trip_distance_id', 'trip_distance']]

def test_trip_distance_dim(trip_distance_dim: pd.DataFrame) -> None:
    assert not trip_distance_dim.isnull().values.any()
    assert all([isinstance(item, int) for item in trip_distance_dim['trip_distance_id']])
    assert all([isinstance(item, float) for item in trip_distance_dim['trip_distance']])    

test_trip_distance_dim(trip_distance_dim)
trip_distance_dim


Modeling the rate_code_dim schema with pandas & testing it

In [ ]:
rate_code_type = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride"
}

rate_code_dim = df[['RatecodeID']].reset_index(drop=True)
rate_code_dim['rate_code_id'] = rate_code_dim.index
rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_type)
rate_code_dim = rate_code_dim[['rate_code_id','RatecodeID','rate_code_name']]

def test_rate_code_dim(rate_code_dim: pd.DataFrame) -> None:
    assert all([isinstance(id, int) for id in rate_code_dim['rate_code_id']])
    assert all([isinstance(id, int) for id in rate_code_dim['RatecodeID']])
    assert all([rate_code in rate_code_type.values() for rate_code in rate_code_dim['rate_code_name']])

test_rate_code_dim(rate_code_dim)
rate_code_dim.head()

Modeling the pickup_location_dim schema with pandas & testing it

In [ ]:
pickup_location_dim = df[['pickup_longitude', 'pickup_latitude']].reset_index(drop=True)
pickup_location_dim['pickup_location_id'] = pickup_location_dim.index
pickup_location_dim = pickup_location_dim[['pickup_location_id','pickup_latitude','pickup_longitude']] 

def test_pickup_location_dim(pickup_location_dim: pd.DataFrame) -> None:
    assert all([isinstance(id, int) for id in pickup_location_dim['pickup_location_id']])
    assert all([isinstance(latitude, float) for latitude in pickup_location_dim['pickup_latitude']])
    assert all([isinstance(longitude, float) for longitude in pickup_location_dim['pickup_longitude']])

test_pickup_location_dim(pickup_location_dim)

pickup_location_dim.head()

Modeling the dropoff_location_dim schema with pandas & testing it

In [ ]:
dropoff_location_dim = df[['dropoff_longitude', 'dropoff_latitude']].reset_index(drop=True)
dropoff_location_dim['dropoff_location_id'] = dropoff_location_dim.index
dropoff_location_dim = dropoff_location_dim[['dropoff_location_id','dropoff_latitude','dropoff_longitude']]

def test_dropoff_location_dim(dropoff_location_dim: pd.DataFrame) -> None:
    assert all([isinstance(id, int) for id in dropoff_location_dim['dropoff_location_id']])
    assert all([isinstance(latitude, float) for latitude in dropoff_location_dim['dropoff_latitude']])
    assert all([isinstance(longitude, float) for longitude in dropoff_location_dim['dropoff_longitude']])

test_dropoff_location_dim(dropoff_location_dim)
dropoff_location_dim.head()

In [ ]:
payment_type_name = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
}

payment_type_dim = df[['payment_type']].reset_index(drop=True)
payment_type_dim['payment_type_id'] = payment_type_dim.index
payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(payment_type_name)
payment_type_dim = payment_type_dim[['payment_type_id', 'payment_type', 'payment_type_name']]


def test_payment_type_dim(payment_type_dim: pd.DataFrame) -> None:
    assert all([isinstance(value, int) for value in payment_type_dim['payment_type']])
    assert all([isinstance(value, int) for value in payment_type_dim['payment_type_id']])
    assert any([payment_type in payment_type_name.values() for payment_type in payment_type_dim['payment_type_name']])

test_payment_type_dim(payment_type_dim)
payment_type_dim





In [ ]:
df.columns
df.head()

Modeling the fact_table schema with pandas, concatenating the dim_tables into & testing it

In [ ]:

# fact_table = df.merge(passenger_count_dim, left_on='trip_id', right_on='passenger_count_id') \
#              .merge(trip_distance_dim, left_on='trip_id', right_on='trip_distance_id') \
#              .merge(rate_code_dim, left_on='trip_id', right_on='rate_code_id') \
#              .merge(pickup_location_dim, left_on='trip_id', right_on='pickup_location_id') \
#              .merge(dropoff_location_dim, left_on='trip_id', right_on='dropoff_location_id')\
#              .merge(datetime_dim, left_on='trip_id', right_on='datetime_id') \
#              .merge(payment_type_dim, left_on='trip_id', right_on='payment_type_id') \
#              [['trip_id','VendorID', 'datetime_id', 'passenger_count_id',
#                'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag', 'pickup_location_id', 'dropoff_location_id',
#                'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
#                'improvement_surcharge', 'total_amount']]

In [ ]:
# fact_table.head()

In [ ]:
# fact_table.columns

In [ ]:
# for value in dict(df.dtypes).values():
#     print(f"\'{str(value).replace('dtype(', '').replace('),', '')}\',")
df_dtypes = [
    'int64',
    'datetime64[ns]',
    'datetime64[ns]',
    'int64',
    'float64',
    'float64',
    'float64',
    'int64',
    'object',
    'float64',
    'float64',
    'int64',
    'float64',
    'float64',
    'float64',
    'float64',
    'float64',
    'float64',
    'float64',
    'int64',
]


In [ ]:
rate_code_dim

In [ ]:
df = pd.read_csv(r'assets/uber_data.csv')
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
df = df.drop_duplicates().reset_index(drop=True)
df['trip_id'] = df.index
df.index.name = "Id"
df['payment_type_name'] = df['payment_type'].map(payment_type_name)
df['rate_code_name'] = df['RatecodeID'].map(rate_code_type)

df = df.dropna()

df.to_csv(r'assets/uber_refined_data.csv')